# Proyecto: Selección de las mejores ubicaciones para pozos petroleros

## **Introducción al proyecto**

En este proyecto, trabajamos para la compañía de extracción de petróleo *OilyGiant*. La empresa busca abrir **200 nuevos pozos de petróleo** en una de tres regiones disponibles. Nuestro objetivo principal es desarrollar un modelo de **regresión lineal** que prediga el **volumen de reservas** en cada pozo, seleccionando las **mejores ubicaciones** para maximizar las ganancias de inversión y minimizar riesgos.

## **Pasos a seguir**

1. **Preparación de datos:** Leeremos y limpiaremos los archivos con datos de los pozos de cada región.  
2. **Entrenamiento del modelo:** Crearemos un modelo de regresión lineal para cada región y evaluaremos su desempeño utilizando métricas como el **RMSE**.  
3. **Selección de los mejores pozos:** Identificaremos los **200 pozos con mayor volumen estimado de reservas** en cada región.  
4. **Evaluación económica:** Calcularemos la ganancia potencial de estos 200 pozos considerando el costo de inversión y los ingresos por barril de petróleo.  
5. **Análisis de riesgos:** Utilizaremos **bootstrapping** para analizar los riesgos de pérdidas y el beneficio promedio.  
6. **Conclusión:** Propondremos la región más adecuada para el desarrollo basándonos en la combinación de beneficio y riesgo.

---

## **Diccionario de datos**

Cada archivo proporcionado (`geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv`) contiene la siguiente información:

- **id:** Identificador único del pozo de petróleo.  
- **f0, f1, f2:** Características numéricas de los puntos. El significado específico de estas variables no es importante, pero influyen en el modelo predictivo.  
- **product:** Volumen de reservas en el pozo de petróleo, expresado en **miles de barriles**.  

---

Este enfoque nos permitirá seleccionar la **región más rentable y menos riesgosa** para el desarrollo de los nuevos pozos. En el proceso, aplicaremos métricas de regresión (como RMSE) y técnicas estadísticas (como **bootstrapping**) para garantizar la fiabilidad de nuestras conclusiones.



# Paso 1 - Carga y Exploración de los Datos

El primer paso para desarrollar nuestro modelo es **cargar y explorar los datos disponibles** de las tres regiones. Esto es crucial para asegurarnos de que toda la información esté correcta y lista para el análisis.

### Objetivos de este paso:
1. **Cargar los conjuntos de datos** proporcionados para las tres regiones.
2. **Verificar** que los datos se hayan importado correctamente.
3. **Inspeccionar la estructura** de los datos (columnas, tipos de datos y valores faltantes).
4. **Separar las variables predictoras** (`f0`, `f1`, `f2`) del valor objetivo (`product`), que representa el volumen de reservas de petróleo en cada pozo.
5. **Explorar estadísticas básicas** para entender las características de los datos y detectar posibles anomalías o valores atípicos.

Con los datos bien organizados y comprendidos, estaremos en una mejor posición para avanzar hacia la construcción y validación del modelo. Además, detectar problemas como valores faltantes o columnas con tipos incorrectos nos permitirá tomar decisiones tempranas y evitar errores en las etapas posteriores.

---

A continuación, cargaremos los archivos de las tres regiones y realizaremos una inspección inicial.


In [1]:
# Paso 1 - Cargar y explorar los datos
import pandas as pd

# Cargar los datos de las tres regiones
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

# Mostrar las primeras filas de cada conjunto de datos para visualización
print("Primeras filas de geo_data_0:")
display(data_0.head())

print("\nPrimeras filas de geo_data_1:")
display(data_1.head())

print("\nPrimeras filas de geo_data_2:")
display(data_2.head())

# Verificación del tamaño de los DataFrames
print("\nTamaño de geo_data_0:", data_0.shape)
print("Tamaño de geo_data_1:", data_1.shape)
print("Tamaño de geo_data_2:", data_2.shape)

# Comprobación de valores ausentes en cada conjunto de datos
print("\nValores ausentes en geo_data_0:")
print(data_0.isna().sum())

print("\nValores ausentes en geo_data_1:")
print(data_1.isna().sum())

print("\nValores ausentes en geo_data_2:")
print(data_2.isna().sum())

# Verificación de tipos de datos
print("\nTipos de datos en geo_data_0:")
print(data_0.dtypes)

print("\nTipos de datos en geo_data_1:")
print(data_1.dtypes)

print("\nTipos de datos en geo_data_2:")
print(data_2.dtypes)

# Comprobación de duplicados
print("\nDuplicados en geo_data_0:", data_0.duplicated().sum())
print("Duplicados en geo_data_1:", data_1.duplicated().sum())
print("Duplicados en geo_data_2:", data_2.duplicated().sum())

# Estadísticas descriptivas para la columna 'product'
print("\nEstadísticas descriptivas de 'product' en geo_data_0:")
display(data_0['product'].describe())

print("\nEstadísticas descriptivas de 'product' en geo_data_1:")
display(data_1['product'].describe())

print("\nEstadísticas descriptivas de 'product' en geo_data_2:")
display(data_2['product'].describe())


Primeras filas de geo_data_0:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Primeras filas de geo_data_1:


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



Primeras filas de geo_data_2:


...

count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64


Estadísticas descriptivas de 'product' en geo_data_1:


count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64


Estadísticas descriptivas de 'product' en geo_data_2:


count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64

# **Resultados de la carga y exploración de los datos**

Tras la exploración de los tres conjuntos de datos proporcionados, estos son los **hallazgos clave**:

1. **Estructura de los datos:** 
   - Cada conjunto de datos tiene **100,000 filas y 5 columnas**. Las columnas incluyen un identificador único (`id`), tres características (`f0`, `f1`, `f2`) y la variable objetivo (`product`), que representa el volumen de reservas de petróleo en miles de barriles.

2. **Valores ausentes:** 
   - No se encontraron **valores faltantes** en ninguno de los tres conjuntos. Esto es positivo, ya que no necesitamos realizar imputaciones ni eliminar filas por datos incompletos.

3. **Duplicados:** 
   - No hay **registros duplicados** en ninguno de los conjuntos. Esto asegura que los datos no estén contaminados por repeticiones innecesarias que podrían sesgar los resultados.

4. **Tipos de datos:** 
   - Las columnas numéricas (`f0`, `f1`, `f2` y `product`) tienen el tipo de dato **float64**, lo que es adecuado para realizar análisis numéricos y modelado predictivo.
   - La columna `id` es de tipo **object**, lo cual es correcto porque es solo un identificador único y no necesita conversión.

5. **Estadísticas descriptivas:** 
   - La media de la columna `product` varía entre regiones: 
     - **Región 0:** Media = 92.5, Máximo = 185.36  
     - **Región 1:** Media = 68.82, Máximo = 137.94  
     - **Región 2:** Media = 95, Máximo = 190.03  
   - Esto sugiere que las reservas de petróleo difieren entre las regiones. La desviación estándar también es similar entre regiones, lo que indica una distribución comparable de los valores de reservas.

---

## **¿Están los datos listos para usarse?**

Los datos están **en buenas condiciones** para comenzar a trabajar:
- No requieren limpieza adicional, ya que no hay valores ausentes ni duplicados.
- Los tipos de datos son adecuados para el análisis.
- Las características (`f0`, `f1`, `f2`) y la variable objetivo (`product`) parecen listas para ser utilizadas en un modelo de regresión lineal.

---

## **Próximo paso**

El siguiente paso será **entrenar y probar el modelo de regresión lineal** en cada región. Dividiremos los datos en conjuntos de entrenamiento y validación (75% y 25%, respectivamente) para evaluar el desempeño del modelo en cada una de las tres regiones. Luego, analizaremos las métricas de precisión, como el **RMSE**, para seleccionar los pozos con el mayor volumen de reservas.



<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>

Excelente trabajo con la carga de la información y su exploración inicial. Como aprendiste el análisisd e duplicados es fundamental para asegurar que no tendremos resultados sesgados. Por otra parte el uso de describe() es muy util cuando mostramos la información de las variables con las que vamos a trabajar. 

# **Paso 2. División de los datos en entrenamiento y validación**

Para poder entrenar y evaluar nuestro modelo de regresión lineal, es necesario **dividir los datos en dos subconjuntos**:  
- **Conjunto de entrenamiento:** Contendrá el **75%** de los datos y se utilizará para ajustar el modelo.
- **Conjunto de validación:** Contendrá el **25%** restante de los datos y se usará para evaluar el rendimiento del modelo una vez entrenado.

Esta división es fundamental para asegurar que el modelo pueda generalizar bien a datos que no ha visto durante el entrenamiento. Si el modelo funciona correctamente en el conjunto de validación, significa que tiene una buena capacidad predictiva y no está sobreajustado (overfitting).

### **Proceso de división**
Utilizaremos la función `train_test_split` de la biblioteca **scikit-learn** para hacer esta división. Esta función separa los datos de forma aleatoria, asegurando que ambas muestras tengan una distribución representativa del conjunto original.

Para cada región (geo_data_0, geo_data_1 y geo_data_2), realizaremos:
1. **Separación de las características** (`f0`, `f1`, `f2`) y la variable objetivo (`product`).
2. **División en entrenamiento y validación** con una proporción **75:25**.
3. **Configuración del parámetro `random_state`** para asegurar que la división sea reproducible en futuras ejecuciones.

Con los datos divididos, podremos avanzar al entrenamiento del modelo de regresión lineal y hacer las predicciones sobre los datos de validación, lo que nos permitirá medir el rendimiento del modelo mediante métricas como el **RMSE**.



In [2]:
# Importación de la función necesaria
from sklearn.model_selection import train_test_split

# División de los datos para geo_data_0
X_0 = data_0[['f0', 'f1', 'f2']]  # Características
y_0 = data_0['product']  # Variable objetivo

X_train_0, X_valid_0, y_train_0, y_valid_0 = train_test_split(
    X_0, y_0, test_size=0.25, random_state=42
)

# División de los datos para geo_data_1
X_1 = data_1[['f0', 'f1', 'f2']]
y_1 = data_1['product']

X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(
    X_1, y_1, test_size=0.25, random_state=42
)

# División de los datos para geo_data_2
X_2 = data_2[['f0', 'f1', 'f2']]
y_2 = data_2['product']

X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(
    X_2, y_2, test_size=0.25, random_state=42
)

# Verificación de las formas de los conjuntos divididos
print("Tamaño de los conjuntos de entrenamiento y validación para geo_data_0:")
print(f"Entrenamiento: {X_train_0.shape}, Validación: {X_valid_0.shape}")

print("\nTamaño de los conjuntos de entrenamiento y validación para geo_data_1:")
print(f"Entrenamiento: {X_train_1.shape}, Validación: {X_valid_1.shape}")

print("\nTamaño de los conjuntos de entrenamiento y validación para geo_data_2:")
print(f"Entrenamiento: {X_train_2.shape}, Validación: {X_valid_2.shape}")




Tamaño de los conjuntos de entrenamiento y validación para geo_data_0:
Entrenamiento: (75000, 3), Validación: (25000, 3)

Tamaño de los conjuntos de entrenamiento y validación para geo_data_1:
Entrenamiento: (75000, 3), Validación: (25000, 3)

Tamaño de los conjuntos de entrenamiento y validación para geo_data_2:
Entrenamiento: (75000, 3), Validación: (25000, 3)


# **Verificación de la división de los datos**

La división de los datos en conjuntos de **entrenamiento y validación** se realizó correctamente. Cada uno de los tres conjuntos de datos fue dividido en una proporción **75:25**, resultando en:
- **75,000** observaciones en el conjunto de entrenamiento.
- **25,000** observaciones en el conjunto de validación.
- **3 características** por conjunto (f0, f1, f2).

Con esta división, estamos listos para proceder al entrenamiento del modelo de regresión lineal y a realizar las predicciones en el conjunto de validación.



# **Entrenamiento del modelo y predicciones para el conjunto de validación**

En este paso, vamos a **entrenar el modelo de regresión lineal** utilizando el conjunto de entrenamiento para cada región y luego **predecir el volumen de reservas** en el conjunto de validación. 

### **Proceso:**
1. **Entrenamiento del modelo:** Utilizaremos el modelo de **Regresión Lineal** de la biblioteca scikit-learn.
2. **Ajuste del modelo:** El modelo se entrenará con las características `f0`, `f1`, `f2` para predecir la variable objetivo `product`.
3. **Predicciones:** Una vez entrenado, el modelo generará **predicciones** para los datos del conjunto de validación.
4. **Evaluación del rendimiento:** Estas predicciones se compararán con los valores reales de la variable `product` en el conjunto de validación para evaluar qué tan preciso es el modelo.

Este paso es crucial para asegurarnos de que el modelo funcione correctamente antes de seleccionar los pozos más prometedores.


In [3]:
# Importar la clase LinearRegression de scikit-learn
from sklearn.linear_model import LinearRegression

# Inicializar el modelo
model_0 = LinearRegression()
model_1 = LinearRegression()
model_2 = LinearRegression()

# Entrenar el modelo para geo_data_0
model_0.fit(X_train_0, y_train_0)
predictions_0 = model_0.predict(X_valid_0)

# Entrenar el modelo para geo_data_1
model_1.fit(X_train_1, y_train_1)
predictions_1 = model_1.predict(X_valid_1)

# Entrenar el modelo para geo_data_2
model_2.fit(X_train_2, y_train_2)
predictions_2 = model_2.predict(X_valid_2)

# Mostrar las primeras 5 predicciones para cada región
print("Primeras 5 predicciones para geo_data_0:", predictions_0[:5])
print("Primeras 5 predicciones para geo_data_1:", predictions_1[:5])
print("Primeras 5 predicciones para geo_data_2:", predictions_2[:5])

# Mostrar el tamaño de las predicciones para verificar que coincidan con los datos de validación
print("\nTamaño de las predicciones para geo_data_0:", predictions_0.shape)
print("Tamaño de las predicciones para geo_data_1:", predictions_1.shape)
print("Tamaño de las predicciones para geo_data_2:", predictions_2.shape)



Primeras 5 predicciones para geo_data_0: [101.90101715  78.21777385 115.26690103 105.61861791  97.9801849 ]
Primeras 5 predicciones para geo_data_1: [ 8.44738063e-01  5.29216119e+01  1.35110385e+02  1.09494863e+02
 -4.72915824e-02]
Primeras 5 predicciones para geo_data_2: [ 98.30191642 101.59246124  52.4490989  109.92212707  72.41184733]

Tamaño de las predicciones para geo_data_0: (25000,)
Tamaño de las predicciones para geo_data_1: (25000,)
Tamaño de las predicciones para geo_data_2: (25000,)


In [4]:
# Guardar las predicciones y las respuestas correctas del conjunto de validación

# Creación de DataFrames para almacenar las predicciones y respuestas reales
results_0 = pd.DataFrame({'predictions': predictions_0, 'actual': y_valid_0.reset_index(drop=True)})
results_1 = pd.DataFrame({'predictions': predictions_1, 'actual': y_valid_1.reset_index(drop=True)})
results_2 = pd.DataFrame({'predictions': predictions_2, 'actual': y_valid_2.reset_index(drop=True)})

# Guardar los DataFrames en archivos CSV para futuras referencias
results_0.to_csv('predictions_geo_data_0.csv', index=False)
results_1.to_csv('predictions_geo_data_1.csv', index=False)
results_2.to_csv('predictions_geo_data_2.csv', index=False)

print("Predicciones y respuestas correctas guardadas en archivos CSV.")


Predicciones y respuestas correctas guardadas en archivos CSV.


# **Guardado de predicciones y respuestas correctas del conjunto de validación**

Hemos creado y guardado **DataFrames** que contienen las **predicciones generadas por el modelo** junto con las **respuestas reales** para los conjuntos de validación de cada una de las tres regiones. Estos DataFrames se han exportado como **archivos CSV** para un uso futuro, asegurando que los resultados estén almacenados de forma organizada y accesible. Esta etapa es fundamental para poder realizar comparaciones entre las predicciones y los valores reales y evaluar el rendimiento del modelo.



# **Volumen medio de reservas predicho y cálculo del RMSE**

El siguiente paso consiste en **evaluar el rendimiento del modelo** de regresión lineal para cada región. Para ello, realizaremos lo siguiente:

1. **Cálculo del volumen medio de reservas predicho:** Calcularemos la media de las predicciones generadas por el modelo para cada región, lo que nos dará una idea de las reservas promedio esperadas según nuestras predicciones.
2. **Cálculo del RMSE (Raíz del Error Cuadrático Medio):** Esta métrica nos indicará qué tan alejadas están, en promedio, las predicciones del modelo respecto a los valores reales. Un **RMSE bajo** significa que las predicciones son precisas, mientras que un RMSE alto indica que el modelo tiene errores significativos.

Esta evaluación nos ayudará a determinar la **calidad del modelo predictivo** y servirá como base para seleccionar las regiones más prometedoras en términos de reservas petroleras.



In [5]:
# Importar la métrica mean_squared_error para el cálculo del RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

# Cálculo del volumen medio de reservas predicho para cada región
mean_prediction_0 = predictions_0.mean()
mean_prediction_1 = predictions_1.mean()
mean_prediction_2 = predictions_2.mean()

print(f"Volumen medio de reservas predicho para geo_data_0: {mean_prediction_0:.2f}")
print(f"Volumen medio de reservas predicho para geo_data_1: {mean_prediction_1:.2f}")
print(f"Volumen medio de reservas predicho para geo_data_2: {mean_prediction_2:.2f}")

# Cálculo del RMSE para cada región
rmse_0 = np.sqrt(mean_squared_error(y_valid_0, predictions_0))
rmse_1 = np.sqrt(mean_squared_error(y_valid_1, predictions_1))
rmse_2 = np.sqrt(mean_squared_error(y_valid_2, predictions_2))

print(f"\nRMSE del modelo para geo_data_0: {rmse_0:.2f}")
print(f"RMSE del modelo para geo_data_1: {rmse_1:.2f}")
print(f"RMSE del modelo para geo_data_2: {rmse_2:.2f}")



Volumen medio de reservas predicho para geo_data_0: 92.40
Volumen medio de reservas predicho para geo_data_1: 68.71
Volumen medio de reservas predicho para geo_data_2: 94.77

RMSE del modelo para geo_data_0: 37.76
RMSE del modelo para geo_data_1: 0.89
RMSE del modelo para geo_data_2: 40.15


# **Análisis de los resultados**

### **Volumen medio de reservas predicho:**
- **Región 0:** 92.40 miles de barriles.
- **Región 1:** 68.71 miles de barriles.
- **Región 2:** 94.77 miles de barriles.

Estos valores nos indican que las **reservas promedio predichas** varían entre las regiones:
- Las **regiones 0 y 2** muestran volúmenes medios de reservas más altos, cercanos a los 92 y 95 miles de barriles, respectivamente.
- La **región 1** tiene el valor más bajo, con un promedio de 68.71 miles de barriles.

### **Análisis del RMSE:**
- **Región 0:** 37.76
- **Región 1:** 0.89
- **Región 2:** 40.15

#### **Interpretación del RMSE:**
- **Región 1** tiene un **RMSE muy bajo** (0.89), lo que sugiere que el modelo predijo con alta precisión los valores reales de reservas. Esto indica que las características (`f0`, `f1`, `f2`) tienen una fuerte correlación con el volumen de reservas en esta región.
- **Región 0** y **Región 2** muestran **valores de RMSE más altos** (37.76 y 40.15, respectivamente), lo que sugiere que las predicciones del modelo tienen un mayor margen de error en estas regiones. Es posible que estas regiones tengan una mayor variabilidad en los datos o que las características disponibles no expliquen tan bien el volumen de reservas.

### **Conclusión:**
- **Región 1** parece ser la más **prometedora en términos de precisión del modelo**. Sin embargo, el volumen medio de reservas predicho es **significativamente menor** que en las otras dos regiones.
- Para seleccionar la región más adecuada, debemos tener en cuenta no solo la precisión del modelo (RMSE) sino también el volumen medio de reservas, ya que **un mayor volumen de reservas implica mayores ingresos potenciales**.
- En los próximos pasos, se realizará una **evaluación más detallada** utilizando técnicas como la selección de los pozos con mayor volumen y el análisis del riesgo mediante **bootstrapping**, lo cual nos ayudará a tomar una decisión informada sobre la región más rentable para el desarrollo.

Este análisis preliminar indica que debemos proceder con cautela y considerar tanto la precisión del modelo como el volumen de reservas para tomar la mejor decisión.



# **Creación de funciones y ejecución de los pasos anteriores para todas las regiones**

En este punto, vamos a **refactorizar el código** que utilizamos en los pasos anteriores, colocando cada proceso en **funciones reutilizables**. Esto nos permitirá:
1. **Evitar duplicación de código:** El mismo flujo se aplicará a cada archivo de datos.
2. **Mejorar la organización del código:** Las funciones permiten estructurar el proceso de manera más clara y eficiente.
3. **Facilitar la repetición del análisis:** Si necesitamos volver a realizar este análisis, podremos hacerlo con facilidad usando las funciones.

### **Proceso dentro de las funciones**
- **Carga y división de los datos:** Separación en entrenamiento y validación.
- **Entrenamiento del modelo y generación de predicciones.**
- **Cálculo del volumen medio de reservas y RMSE.**
- **Almacenamiento de las predicciones y respuestas en CSV.**
- **Análisis de los resultados.**

Este enfoque permite aplicar **el mismo procedimiento de forma coherente** a los tres conjuntos de datos (`geo_data_0.csv`, `geo_data_1.csv` y `geo_data_2.csv`) para que podamos comparar los resultados entre las regiones y tomar decisiones basadas en datos consistentes.


In [6]:
# Importar las bibliotecas necesarias
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# Función para cargar los datos y dividirlos en entrenamiento y validación
def load_and_split_data(file_path):
    data = pd.read_csv(file_path)
    X = data[['f0', 'f1', 'f2']]
    y = data['product']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    return X_train, X_valid, y_train, y_valid

# Función para entrenar el modelo y generar predicciones
def train_and_predict(X_train, y_train, X_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    return predictions

# Función para calcular el volumen medio de reservas y el RMSE
def calculate_metrics(y_valid, predictions):
    mean_predicted = predictions.mean()
    rmse = np.sqrt(mean_squared_error(y_valid, predictions))
    return mean_predicted, rmse

# Función para guardar predicciones y respuestas correctas
def save_results(predictions, y_valid, region_name):
    results = pd.DataFrame({'predictions': predictions, 'actual': y_valid.reset_index(drop=True)})
    results.to_csv(f'predictions_{region_name}.csv', index=False)
    print(f"Predicciones y respuestas guardadas para {region_name}")

# Función para ejecutar todo el proceso de una región
def process_region(file_path, region_name):
    # Cargar y dividir los datos
    X_train, X_valid, y_train, y_valid = load_and_split_data(file_path)
    
    # Entrenar el modelo y obtener predicciones
    predictions = train_and_predict(X_train, y_train, X_valid)
    
    # Calcular el volumen medio y RMSE
    mean_predicted, rmse = calculate_metrics(y_valid, predictions)
    
    # Guardar los resultados
    save_results(predictions, y_valid, region_name)
    
    # Mostrar los resultados
    print(f"Resultados para {region_name}:")
    print(f"Volumen medio de reservas predicho: {mean_predicted:.2f}")
    print(f"RMSE: {rmse:.2f}\n")
    
    return mean_predicted, rmse

# Ejecutar para cada conjunto de datos
process_region('/datasets/geo_data_0.csv', 'geo_data_0')
process_region('/datasets/geo_data_1.csv', 'geo_data_1')
process_region('/datasets/geo_data_2.csv', 'geo_data_2')



Predicciones y respuestas guardadas para geo_data_0
Resultados para geo_data_0:
Volumen medio de reservas predicho: 92.40
RMSE: 37.76

Predicciones y respuestas guardadas para geo_data_1
Resultados para geo_data_1:
Volumen medio de reservas predicho: 68.71
RMSE: 0.89

Predicciones y respuestas guardadas para geo_data_2
Resultados para geo_data_2:
Volumen medio de reservas predicho: 94.77
RMSE: 40.15



(94.77102387765939, 40.14587231134218)

# **Resultados del análisis para todas las regiones**

Hemos ejecutado el flujo completo para las tres regiones, obteniendo los siguientes resultados:

### **Resultados por región:**
- **Región 0:**
  - Volumen medio de reservas predicho: **92.40 miles de barriles**
  - RMSE: **37.76**

- **Región 1:**
  - Volumen medio de reservas predicho: **68.71 miles de barriles**
  - RMSE: **0.89**

- **Región 2:**
  - Volumen medio de reservas predicho: **94.77 miles de barriles**
  - RMSE: **40.15**

### **Interpretación de los resultados:**
- **Región 1** destaca por tener un **RMSE muy bajo (0.89)**, lo que indica que el modelo de regresión lineal predice con alta precisión en esta región.
- Sin embargo, **las regiones 0 y 2** presentan **volúmenes medios de reservas más altos** (92.40 y 94.77 miles de barriles, respectivamente), a pesar de tener RMSE más elevados.
- El **volumen medio de reservas** es un factor importante a tener en cuenta, ya que implica **potencial de rentabilidad**, pero también es necesario considerar la **precisión del modelo** en cada región para minimizar riesgos.

Estos resultados nos indican que:
1. La **Región 1** tiene predicciones más precisas, pero menor volumen medio de reservas.
2. Las **Regiones 0 y 2** tienen mayores volúmenes de reservas, pero con más variabilidad en las predicciones.

Este análisis preliminar sugiere que las **Regiones 0 y 2** podrían ser más rentables, pero aún necesitamos evaluar el riesgo y la ganancia potencial para tomar la mejor decisión sobre dónde desarrollar los pozos.



# **Preparación para el cálculo de ganancias**

En este paso, nos **preparamos para calcular las ganancias** estimadas en cada región. Este análisis será crucial para determinar **qué región tiene el mayor potencial de rentabilidad** y así tomar decisiones informadas sobre el desarrollo de pozos. 

### **Proceso:**
1. **Almacenamiento de valores clave:** 
   - Guardaremos en **variables separadas** todos los valores necesarios para el cálculo de las ganancias, incluyendo:
     - **Presupuesto total** disponible para el desarrollo.
     - **Costo por pozo** y **número de pozos**.
     - **Precio por unidad de producto (barril).**
     - **Predicciones de volumen de reservas** para cada región.
     - **Cantidad mínima de reservas** necesarias para evitar pérdidas.

2. **Importancia del almacenamiento organizado:** 
   - Mantener estos valores en variables separadas facilitará los cálculos posteriores y mejorará la legibilidad del código.
   - Esto nos permitirá trabajar eficientemente en el análisis de riesgos y el cálculo de ganancias usando técnicas como la **selección de pozos con mayor volumen** y el **bootstrapping**.

Este es un paso importante antes de avanzar al análisis de ganancias y riesgos, ya que nos asegura que todos los datos y parámetros estén bien organizados y listos para los cálculos.



In [7]:
# Definir los valores clave para el cálculo de ganancias

# Presupuesto total disponible en dólares
total_budget = 100_000_000  # 100 millones de dólares

# Número de pozos a desarrollar
n_pozos = 200

# Precio por barril de petróleo en dólares
price_per_barrel = 4.5

# Ingresos por unidad de producto (miles de barriles)
income_per_unit = 4_500  # 4.5 USD * 1000 barriles

# Cantidad mínima de reservas necesarias por pozo para evitar pérdidas (en unidades)
min_reserves_per_pozo = 500_000 / income_per_unit  # 500,000 USD por pozo / 4,500 USD por unidad
print(f"Cantidad mínima de reservas necesarias por pozo: {min_reserves_per_pozo:.2f} unidades")

# Cargar las predicciones guardadas para cada región
predictions_0 = pd.read_csv('predictions_geo_data_0.csv')['predictions']
predictions_1 = pd.read_csv('predictions_geo_data_1.csv')['predictions']
predictions_2 = pd.read_csv('predictions_geo_data_2.csv')['predictions']

# Verificar las primeras 5 predicciones para asegurarse de que se cargaron correctamente
print(f"Primeras 5 predicciones para geo_data_0: {predictions_0[:5].values}")
print(f"Primeras 5 predicciones para geo_data_1: {predictions_1[:5].values}")
print(f"Primeras 5 predicciones para geo_data_2: {predictions_2[:5].values}")



Cantidad mínima de reservas necesarias por pozo: 111.11 unidades
Primeras 5 predicciones para geo_data_0: [101.90101715  78.21777385 115.26690103 105.61861791  97.9801849 ]
Primeras 5 predicciones para geo_data_1: [ 8.44738063e-01  5.29216119e+01  1.35110385e+02  1.09494863e+02
 -4.72915824e-02]
Primeras 5 predicciones para geo_data_2: [ 98.30191642 101.59246124  52.4490989  109.92212707  72.41184733]


# **Resultados de la preparación para el cálculo de ganancias**

Hemos almacenado correctamente los **valores necesarios** para el cálculo de ganancias, incluyendo:
- **Presupuesto total:** 100 millones de dólares.
- **Número de pozos:** 200.
- **Precio por barril:** 4.5 USD.
- **Reservas mínimas necesarias:** Cada pozo debe generar al menos **111.11 unidades** para evitar pérdidas.

Además, hemos verificado que las **predicciones se cargaron correctamente** para cada región, lo que nos asegura que estamos listos para proceder con el cálculo de las ganancias.

---

Podemos continuar con el siguiente paso, donde compararemos las **reservas mínimas necesarias** con el **volumen medio de reservas** en cada región para evaluar si alguna región está por debajo del umbral de rentabilidad. 


In [8]:
# Valores mínimos necesarios por pozo para evitar pérdidas
min_units_required = 111.11

# Cálculo de la media de reservas en cada región (en unidades de producto)
mean_reserves_0 = predictions_0.mean()
mean_reserves_1 = predictions_1.mean()
mean_reserves_2 = predictions_2.mean()

# Mostrar los resultados y compararlos con la cantidad mínima necesaria
print(f"Volumen medio de reservas en geo_data_0: {mean_reserves_0:.2f} unidades")
print(f"¿Supera el umbral? {'Sí' if mean_reserves_0 >= min_units_required else 'No'}\n")

print(f"Volumen medio de reservas en geo_data_1: {mean_reserves_1:.2f} unidades")
print(f"¿Supera el umbral? {'Sí' if mean_reserves_1 >= min_units_required else 'No'}\n")

print(f"Volumen medio de reservas en geo_data_2: {mean_reserves_2:.2f} unidades")
print(f"¿Supera el umbral? {'Sí' if mean_reserves_2 >= min_units_required else 'No'}\n")


Volumen medio de reservas en geo_data_0: 92.40 unidades
¿Supera el umbral? No

Volumen medio de reservas en geo_data_1: 68.71 unidades
¿Supera el umbral? No

Volumen medio de reservas en geo_data_2: 94.77 unidades
¿Supera el umbral? No



# **Análisis de los resultados**

### **Volumen medio de reservas por región:**
- **Región 0:** 92.40 unidades  
- **Región 1:** 68.71 unidades  
- **Región 2:** 94.77 unidades  

### **Comparación con el umbral de rentabilidad (111.11 unidades):**
- **Región 0:** No supera el umbral.
- **Región 1:** No supera el umbral.
- **Región 2:** No supera el umbral.

### **Interpretación:**
- Ninguna de las tres regiones tiene un **volumen medio de reservas predicho** que supere el **umbral de 111.11 unidades** necesario para evitar pérdidas.
- Esto sugiere que, en promedio, los pozos en estas regiones **no generarían los 500,000 USD mínimos** por pozo para evitar pérdidas.

### **Conclusión preliminar:**
Aunque los promedios no alcanzan el umbral mínimo de rentabilidad, todavía existe la posibilidad de lograr beneficios seleccionando **los 200 pozos con los mayores volúmenes predichos** en cada región. La siguiente etapa consistirá en identificar los **mejores pozos** para analizar si, con una selección óptima, podemos alcanzar la rentabilidad deseada.

# **Conclusiones sobre la preparación para el cálculo del beneficio**

Con base en el análisis realizado, hemos concluido lo siguiente:

1. **Volumen medio de reservas:**  
   - Ninguna de las tres regiones tiene un volumen medio predicho que supere el umbral de **111.11 unidades** por pozo, lo que significa que no alcanzan los **500,000 USD** mínimos por pozo para evitar pérdidas.  
   - Esto sugiere que no podemos depender del volumen medio, y será crucial **seleccionar los pozos con los mayores volúmenes predichos** en cada región para alcanzar la rentabilidad deseada.

2. **Siguiente paso para el cálculo del beneficio:**  
   - Seleccionaremos los **200 pozos con los mayores volúmenes de reservas** en cada región. Esta selección nos permitirá concentrarnos en los pozos más rentables y evitar aquellos con reservas menores que generen pérdidas.
   - Posteriormente, realizaremos un **análisis de ganancias y riesgos** mediante la técnica de **bootstrapping**, lo que nos dará una visión más completa del beneficio potencial y la probabilidad de pérdidas.

3. **Preparación final:**  
   - Con los valores económicos (presupuesto, número de pozos, precio por unidad, y reservas mínimas) ya almacenados y las predicciones cargadas correctamente, estamos listos para proceder al **cálculo de las ganancias y la selección de pozos**.

Resumiendo, aunque ninguna región cumple con el umbral de rentabilidad a nivel promedio, tenemos los elementos listos para avanzar y concentrar el desarrollo en los pozos más prometedores.


# **Selección de los 200 pozos más prometedores en cada región**

En este paso, nos enfocaremos en **identificar los 200 pozos con las mayores reservas predichas** en cada una de las tres regiones. Esta selección es crucial porque:
- Nos permitirá concentrarnos en los **pozos más rentables** y evitar aquellos con reservas insuficientes.
- A pesar de que el volumen medio de reservas por pozo no alcanza el umbral de rentabilidad, al seleccionar los pozos con **las mayores reservas estimadas**, aumentamos la posibilidad de lograr el beneficio deseado.

### **Proceso:**
1. **Ordenar las predicciones:** Ordenaremos las predicciones de cada región de mayor a menor.
2. **Seleccionar los 200 pozos más altos:** Tomaremos los primeros 200 valores de la lista ordenada para cada región.
3. **Almacenar los pozos seleccionados:** Los pozos seleccionados se guardarán para su posterior análisis de ganancias y riesgos.

Este paso nos acerca a calcular las **ganancias potenciales**, ya que basaremos el análisis en los pozos más prometedores en términos de reservas predichas.


In [9]:
# Función para seleccionar los 200 pozos con las mayores reservas predichas
def select_top_200(predictions):
    # Ordenar las predicciones de mayor a menor y seleccionar las primeras 200
    top_200 = predictions.sort_values(ascending=False).head(200)
    return top_200

# Seleccionar los 200 pozos más prometedores para cada región
top_200_0 = select_top_200(predictions_0)
top_200_1 = select_top_200(predictions_1)
top_200_2 = select_top_200(predictions_2)

# Mostrar las primeras 5 predicciones seleccionadas para verificar el proceso
print("Primeras 5 predicciones seleccionadas para geo_data_0:")
print(top_200_0.head())

print("\nPrimeras 5 predicciones seleccionadas para geo_data_1:")
print(top_200_1.head())

print("\nPrimeras 5 predicciones seleccionadas para geo_data_2:")
print(top_200_2.head())

# Guardar las selecciones en archivos CSV para futuras referencias
top_200_0.to_csv('top_200_geo_data_0.csv', index=False)
top_200_1.to_csv('top_200_geo_data_1.csv', index=False)
top_200_2.to_csv('top_200_geo_data_2.csv', index=False)

print("\nSelección de los 200 pozos más prometedores completada y guardada.")


Primeras 5 predicciones seleccionadas para geo_data_0:
6958     176.536104
18194    176.274510
17251    173.249504
457      172.802708
2202     172.744977
Name: predictions, dtype: float64

Primeras 5 predicciones seleccionadas para geo_data_1:
20776    139.983277
2323     139.700803
13895    139.616544
6950     139.514768
9151     139.472212
Name: predictions, dtype: float64

Primeras 5 predicciones seleccionadas para geo_data_2:
21852    170.529209
10722    169.673332
6209     165.300724
8203     164.613896
8042     163.964000
Name: predictions, dtype: float64

Selección de los 200 pozos más prometedores completada y guardada.


# **Resumen del volumen objetivo de reservas para los 200 pozos seleccionados**

En este paso, resumiremos el **volumen total de reservas predicho** para los **200 pozos más prometedores** en cada una de las tres regiones. Esta información es clave para estimar las ganancias potenciales de cada región y comparar su rentabilidad.

### **Proceso:**
1. **Suma del volumen de reservas predicho:** Calcularemos el volumen total de las reservas para los **200 pozos seleccionados** en cada región.
2. **Almacenamiento del resumen:** Guardaremos estos volúmenes totales como parte del análisis para que puedan ser utilizados en el cálculo de ganancias y el análisis de riesgo posterior.

Este resumen nos ayudará a identificar qué región tiene **el mayor potencial de reservas concentradas en los mejores pozos** y será un elemento fundamental en la **selección de la región más prometedora** para el desarrollo.


In [10]:
# Función para calcular el volumen total de reservas de los 200 pozos seleccionados
def summarize_reserves(top_200):
    total_reserves = top_200.sum()
    return total_reserves

# Calcular el volumen total de reservas para cada región
total_reserves_0 = summarize_reserves(top_200_0)
total_reserves_1 = summarize_reserves(top_200_1)
total_reserves_2 = summarize_reserves(top_200_2)

# Mostrar los resultados
print(f"Volumen total de reservas en los 200 pozos de geo_data_0: {total_reserves_0:.2f} unidades")
print(f"Volumen total de reservas en los 200 pozos de geo_data_1: {total_reserves_1:.2f} unidades")
print(f"Volumen total de reservas en los 200 pozos de geo_data_2: {total_reserves_2:.2f} unidades")

# Guardar los resultados en archivos CSV
pd.DataFrame({'total_reserves': [total_reserves_0]}).to_csv('total_reserves_geo_data_0.csv', index=False)
pd.DataFrame({'total_reserves': [total_reserves_1]}).to_csv('total_reserves_geo_data_1.csv', index=False)
pd.DataFrame({'total_reserves': [total_reserves_2]}).to_csv('total_reserves_geo_data_2.csv', index=False)

print("\nResumen del volumen de reservas completado y guardado.")


Volumen total de reservas en los 200 pozos de geo_data_0: 30881.46 unidades
Volumen total de reservas en los 200 pozos de geo_data_1: 27748.75 unidades
Volumen total de reservas en los 200 pozos de geo_data_2: 29728.85 unidades

Resumen del volumen de reservas completado y guardado.


# **Resumen del volumen total de reservas en los 200 pozos seleccionados**

### **Resultados por región:**
- **Región 0:** 30,881.46 unidades  
- **Región 1:** 27,748.75 unidades  
- **Región 2:** 29,728.85 unidades  

### **Interpretación:**
- **Región 0** destaca por tener el mayor volumen total de reservas entre los 200 pozos más prometedores, lo que indica un mayor potencial de rentabilidad.
- **Región 1** tiene el volumen más bajo de los tres, lo que podría sugerir una opción menos competitiva.
- **Región 2** presenta un volumen total considerable, cercano al de la Región 0, lo que la convierte en otra opción viable.

Con base en estos resultados, **Región 0 y Región 2** parecen las más prometedoras, ya que concentran un alto volumen de reservas en sus mejores pozos. Sin embargo, todavía es necesario calcular las **ganancias potenciales** para cada región. Este análisis nos permitirá seleccionar la **región más rentable para el desarrollo de pozos petrolíferos** y justificar nuestra decisión basada en los beneficios económicos esperados.



# **Cálculo de la ganancia potencial para los 200 pozos principales por región**

En este paso, calcularemos la **ganancia potencial** de los 200 pozos seleccionados en cada región. Este análisis nos permitirá identificar **qué región generaría los mayores beneficios económicos** si se desarrollaran los pozos con las mayores reservas predichas.

### **Proceso:**
1. **Fórmula del cálculo de ganancias:**  
   - La ganancia total se calculará como:
     \[
     \text{Ganancia} = \text{Volumen Total de Reservas} \times \text{Ingresos por Unidad}
     \]
     Donde:
     - **Volumen Total de Reservas:** La suma de las reservas de los 200 pozos seleccionados.
     - **Ingresos por Unidad:** 4,500 USD por cada 1,000 barriles (1 unidad).

2. **Comparación entre regiones:**  
   - Calcularemos la ganancia para cada región con base en sus 200 pozos más prometedores y **compararemos los resultados** para determinar cuál es la opción más rentable.

3. **Selección de la región más rentable:**  
   - Con los resultados obtenidos, presentaremos una recomendación sobre **qué región es la mejor opción** para el desarrollo de pozos, justificando la decisión con base en las ganancias proyectadas.

Este análisis será fundamental para asegurar que los recursos se inviertan en la región con **mayor potencial de retorno económico**, minimizando riesgos y maximizando beneficios.


In [11]:
# Función para calcular la ganancia total para los 200 pozos seleccionados
def calculate_profit(total_reserves, income_per_unit=4500):
    return total_reserves * income_per_unit

# Calcular la ganancia potencial para cada región
profit_0 = calculate_profit(total_reserves_0)
profit_1 = calculate_profit(total_reserves_1)
profit_2 = calculate_profit(total_reserves_2)

# Mostrar las ganancias potenciales por región
print(f"Ganancia potencial de los 200 pozos en geo_data_0: ${profit_0:,.2f}")
print(f"Ganancia potencial de los 200 pozos en geo_data_1: ${profit_1:,.2f}")
print(f"Ganancia potencial de los 200 pozos en geo_data_2: ${profit_2:,.2f}")

# Seleccionar la región con la mayor ganancia
profits = {'Región 0': profit_0, 'Región 1': profit_1, 'Región 2': profit_2}
best_region = max(profits, key=profits.get)

print(f"\nLa mejor opción es {best_region} con una ganancia proyectada de ${profits[best_region]:,.2f}.")


Ganancia potencial de los 200 pozos en geo_data_0: $138,966,584.80
Ganancia potencial de los 200 pozos en geo_data_1: $124,869,381.15
Ganancia potencial de los 200 pozos en geo_data_2: $133,779,815.14

La mejor opción es Región 0 con una ganancia proyectada de $138,966,584.80.


# **Conclusiones sobre la ganancia potencial y selección de la región más prometedora**

### **Ganancia potencial por región:**
- **Región 0:** $138,966,584.80  
- **Región 1:** $124,869,381.15  
- **Región 2:** $133,779,815.14  

### **Análisis:**
- La **Región 0** presenta la **mayor ganancia proyectada** entre las tres regiones, con $138,966,584.80, lo que la convierte en la opción más rentable para el desarrollo de los 200 pozos seleccionados.
- **Región 2** también muestra un desempeño competitivo, con una ganancia proyectada cercana a la de la Región 0. Sin embargo, la diferencia en la ganancia hace que **Región 0** sea una opción ligeramente más favorable.
- **Región 1**, aunque presenta un modelo con alta precisión (bajo RMSE), genera la menor ganancia de las tres, lo que sugiere que **el potencial económico es más limitado** en comparación con las otras regiones.

### **Recomendación hasta este punto:**
Se recomienda proceder con el **desarrollo de los 200 pozos seleccionados en la Región 0**. Esta región no solo ofrece el **mayor volumen total de reservas** en los pozos más prometedores, sino que también proyecta la **mayor ganancia económica**. Invertir en esta región maximiza la probabilidad de alcanzar un retorno significativo sobre la inversión, asegurando el uso más eficiente de los recursos disponibles.

Esta elección está respaldada por el análisis detallado de las predicciones, el volumen de reservas y la rentabilidad proyectada. Aunque las otras regiones tienen potencial, **Región 0** es la opción más atractiva desde una perspectiva financiera.


# **Cálculo de riesgos y distribución de ganancias mediante bootstrapping**

En este punto, utilizaremos la técnica de **bootstrapping** para evaluar la **distribución de los beneficios** en cada región. Este método nos permitirá:
1. **Simular la variabilidad en las ganancias:** Tomaremos **1,000 muestras aleatorias** con reemplazo de los 200 pozos seleccionados en cada región.
2. **Calcular la ganancia para cada muestra:** Esto nos proporcionará una **distribución de las ganancias** y nos permitirá observar posibles escenarios favorables o desfavorables.
3. **Evaluar riesgos:** 
   - Veremos la **probabilidad de pérdidas**, es decir, cuántas veces una muestra generó menos de los **500,000 USD por pozo** necesarios para evitar pérdidas.
   - Esto nos dará una visión más clara del **riesgo asociado** con cada región, complementando el análisis de rentabilidad realizado previamente.

### **Importancia del bootstrapping:**
Esta técnica es fundamental para analizar **riesgos** y no solo ganancias potenciales. Nos permite estimar con mayor certeza qué tan estable y predecible es la ganancia en cada región, lo que es crucial para la toma de decisiones estratégicas.

Con este análisis, podremos presentar una recomendación final basada en tanto la **ganancia proyectada** como el **riesgo de pérdida**.


In [12]:
import numpy as np

# Función para realizar bootstrapping y calcular la distribución de ganancias
def bootstrap_profit(predictions, n_samples=1000, n_pozos=200, income_per_unit=4500):
    profits = []
    
    # Generar 1000 muestras aleatorias con reemplazo
    for _ in range(n_samples):
        sample = predictions.sample(n=n_pozos, replace=True)
        total_reserves = sample.sum()
        profit = total_reserves * income_per_unit
        profits.append(profit)
    
    return np.array(profits)

# Realizar bootstrapping para cada región
profits_0 = bootstrap_profit(top_200_0)
profits_1 = bootstrap_profit(top_200_1)
profits_2 = bootstrap_profit(top_200_2)

# Calcular estadísticas descriptivas para cada región
print(f"Media de ganancias para geo_data_0: ${profits_0.mean():,.2f}")
print(f"Media de ganancias para geo_data_1: ${profits_1.mean():,.2f}")
print(f"Media de ganancias para geo_data_2: ${profits_2.mean():,.2f}")

# Calcular la probabilidad de pérdida (ganancia negativa) en cada región
loss_threshold = 500_000 * 200  # 500,000 USD por pozo para 200 pozos
loss_prob_0 = np.mean(profits_0 < loss_threshold)
loss_prob_1 = np.mean(profits_1 < loss_threshold)
loss_prob_2 = np.mean(profits_2 < loss_threshold)

print(f"\nProbabilidad de pérdida para geo_data_0: {loss_prob_0:.2%}")
print(f"Probabilidad de pérdida para geo_data_1: {loss_prob_1:.2%}")
print(f"Probabilidad de pérdida para geo_data_2: {loss_prob_2:.2%}")


Media de ganancias para geo_data_0: $138,969,541.65
Media de ganancias para geo_data_1: $124,869,404.87
Media de ganancias para geo_data_2: $133,766,239.06

Probabilidad de pérdida para geo_data_0: 0.00%
Probabilidad de pérdida para geo_data_1: 0.00%
Probabilidad de pérdida para geo_data_2: 0.00%


# **Resultados del análisis de ganancias y riesgos mediante bootstrapping**

### **Media de ganancias por región:**
- **Región 0:** $138,969,541.65  
- **Región 1:** $124,869,404.87  
- **Región 2:** $133,766,239.06  

### **Probabilidad de pérdida:**
- **Región 0:** 0.00%  
- **Región 1:** 0.00%  
- **Región 2:** 0.00%  

### **Interpretación:**
1. **Rentabilidad proyectada:**  
   - La **Región 0** sigue destacando como la opción más rentable, con una media de ganancias cercana a los **138.97 millones de USD**.
   - **Región 2** también se mantiene como una opción competitiva, con ganancias cercanas a **133.77 millones de USD**.
   - **Región 1**, aunque con la menor ganancia proyectada, ofrece un rendimiento significativo con **124.87 millones de USD**.

2. **Riesgo de pérdida:**  
   - La probabilidad de pérdida es **0.00%** en todas las regiones. Esto indica que, en cada una de las 1,000 simulaciones realizadas, las ganancias proyectadas siempre superaron el umbral mínimo necesario de **500,000 USD por pozo**, lo que refleja un **riesgo mínimo** en las inversiones.

### **Conclusión:**
- Con base en la **media de ganancias** y la **ausencia de riesgo de pérdida**, **Región 0** se mantiene como la **mejor opción** para el desarrollo de los pozos petrolíferos. No solo presenta la mayor rentabilidad proyectada, sino que también ofrece estabilidad en los retornos, con un riesgo prácticamente nulo.
- **Región 2** podría considerarse como una opción alternativa, dado que tiene una ganancia competitiva. Sin embargo, la mayor rentabilidad de la **Región 0** la hace la opción más favorable para maximizar el retorno de inversión.

La combinación de **rentabilidad** y **riesgo mínimo** respalda la decisión de desarrollar los 200 pozos en la **Región 0**, maximizando las ganancias y garantizando estabilidad en los retornos.



# **Cálculo del beneficio promedio, intervalo de confianza del 95% y riesgo de pérdidas**

En este paso, realizaremos un análisis más detallado del rendimiento económico de cada región. Nos enfocaremos en:
1. **Beneficio promedio:** Confirmaremos el promedio de las ganancias obtenidas en las simulaciones de bootstrapping.
2. **Intervalo de confianza del 95%:** Calcularemos el intervalo de confianza para las ganancias, lo que nos permitirá entender la **variabilidad** de los beneficios y asegurar que los retornos proyectados son confiables.
3. **Riesgo de pérdidas:** 
   - El **riesgo de pérdidas** se expresa como la **probabilidad de obtener ganancias negativas** o menores al umbral mínimo de 500,000 USD por pozo.
   - Este porcentaje se calcula en función de las **muestras generadas mediante bootstrapping**, proporcionando una medida objetiva del riesgo para cada región.

### **Importancia del análisis:**
Este análisis es crucial para tomar decisiones informadas. No solo consideraremos el beneficio promedio, sino también la **dispersión de los resultados** y la **probabilidad de pérdidas**. Esta combinación de métricas garantizará que elegimos la región con el **mejor equilibrio entre rentabilidad y riesgo**.


In [13]:
# Función para calcular el intervalo de confianza del 95% y el beneficio promedio
def calculate_statistics(profits):
    mean_profit = profits.mean()
    lower_bound = np.percentile(profits, 2.5)
    upper_bound = np.percentile(profits, 97.5)
    return mean_profit, (lower_bound, upper_bound)

# Calcular estadísticas para cada región
mean_0, conf_interval_0 = calculate_statistics(profits_0)
mean_1, conf_interval_1 = calculate_statistics(profits_1)
mean_2, conf_interval_2 = calculate_statistics(profits_2)

# Mostrar los resultados
print(f"Beneficio promedio para geo_data_0: ${mean_0:,.2f}")
print(f"Intervalo de confianza del 95% para geo_data_0: ${conf_interval_0[0]:,.2f} - ${conf_interval_0[1]:,.2f}")

print(f"\nBeneficio promedio para geo_data_1: ${mean_1:,.2f}")
print(f"Intervalo de confianza del 95% para geo_data_1: ${conf_interval_1[0]:,.2f} - ${conf_interval_1[1]:,.2f}")

print(f"\nBeneficio promedio para geo_data_2: ${mean_2:,.2f}")
print(f"Intervalo de confianza del 95% para geo_data_2: ${conf_interval_2[0]:,.2f} - ${conf_interval_2[1]:,.2f}")

# Recalcular probabilidad de pérdidas (menores a 500,000 USD por pozo)
loss_threshold = 500_000 * 200  # 500,000 USD por pozo para 200 pozos

loss_prob_0 = np.mean(profits_0 < loss_threshold)
loss_prob_1 = np.mean(profits_1 < loss_threshold)
loss_prob_2 = np.mean(profits_2 < loss_threshold)

# Mostrar probabilidad de pérdida para cada región
print(f"\nProbabilidad de pérdida para geo_data_0: {loss_prob_0:.2%}")
print(f"Probabilidad de pérdida para geo_data_1: {loss_prob_1:.2%}")
print(f"Probabilidad de pérdida para geo_data_2: {loss_prob_2:.2%}")


Beneficio promedio para geo_data_0: $138,969,541.65
Intervalo de confianza del 95% para geo_data_0: $138,187,810.86 - $139,788,866.49

Beneficio promedio para geo_data_1: $124,869,404.87
Intervalo de confianza del 95% para geo_data_1: $124,834,611.49 - $124,903,907.52

Beneficio promedio para geo_data_2: $133,766,239.06
Intervalo de confianza del 95% para geo_data_2: $133,078,525.76 - $134,474,850.42

Probabilidad de pérdida para geo_data_0: 0.00%
Probabilidad de pérdida para geo_data_1: 0.00%
Probabilidad de pérdida para geo_data_2: 0.00%


# **Resultados del análisis de beneficios y riesgos**

### **Beneficio promedio por región:**
- **Región 0:** $138,969,541.65  
- **Región 1:** $124,869,404.87  
- **Región 2:** $133,766,239.06  

### **Intervalo de confianza del 95% para las ganancias:**
- **Región 0:** $138,187,810.86 - $139,788,866.49  
- **Región 1:** $124,834,611.49 - $124,903,907.52  
- **Región 2:** $133,078,525.76 - $134,474,850.42  

### **Probabilidad de pérdida:**
- **Región 0:** 0.00%  
- **Región 1:** 0.00%  
- **Región 2:** 0.00%  

### **Interpretación:**
1. **Beneficio promedio y estabilidad:**  
   - La **Región 0** sigue siendo la más prometedora, con un beneficio promedio cercano a **138.97 millones de USD** y un intervalo de confianza estrecho que refleja **poca variabilidad** en las ganancias.
   - **Región 1** tiene la menor ganancia proyectada, pero también muestra un intervalo de confianza muy ajustado, indicando **resultados consistentes**.
   - **Región 2** presenta un beneficio competitivo, con un intervalo de confianza que sugiere **ligera variabilidad** en los resultados.

2. **Riesgo de pérdida:**  
   - Ninguna de las tres regiones tiene **probabilidad de pérdida**, lo que indica que todas las simulaciones generaron ganancias mayores al umbral mínimo. Esto refleja un **riesgo mínimo** para la inversión en cualquiera de las regiones.

### **Conclusión:**
- La **Región 0** es la **mejor opción** para el desarrollo de los pozos, ya que ofrece la mayor rentabilidad proyectada, estabilidad en los resultados y un riesgo nulo.
- **Región 2** podría considerarse como una alternativa viable, pero la **Región 0** sigue siendo la opción más atractiva en términos económicos.

Este análisis final confirma que la inversión en **Región 0** maximiza el retorno con el menor riesgo posible, lo que respalda la decisión de priorizar el desarrollo de pozos en esta región.



# **Conclusiones finales y recomendación para el desarrollo de pozos petrolíferos**

### **Análisis y resultados finales:**
- **Región 0** se mantiene como la **mejor opción** tanto en términos de beneficio proyectado como de estabilidad en los resultados:
  - **Beneficio promedio:** $138,969,541.65  
  - **Intervalo de confianza del 95%:** $138,187,810.86 - $139,788,866.49  
  - **Riesgo de pérdida:** 0.00%

- **Región 2** también es competitiva, con un beneficio proyectado cercano, pero **Región 0** sigue siendo superior en términos de **rentabilidad y consistencia**.

- **Región 1** presenta una menor rentabilidad proyectada, aunque con resultados consistentes y riesgo nulo. Sin embargo, sus ganancias no superan a las de las otras dos regiones, por lo que no se recomienda priorizar esta opción.

### **Recomendación final:**
Se recomienda desarrollar los **200 pozos seleccionados en la Región 0**. Esta región ofrece:
1. **Mayor rentabilidad:** La ganancia proyectada es la más alta entre las tres regiones.
2. **Estabilidad en los resultados:** El intervalo de confianza del 95% es estrecho, lo que refleja poca variabilidad en las ganancias.
3. **Riesgo mínimo:** No existe riesgo de pérdida, asegurando un retorno positivo sobre la inversión.

### **¿Coincide la elección con la decisión anterior?**
Sí, nuestra **selección final coincide con la decisión anterior** realizada en el análisis previo. En ambos casos, **Región 0** se destacó como la opción más prometedora, gracias a su combinación de **alta rentabilidad y riesgo mínimo**.

Esta recomendación asegura que los recursos se inviertan de manera eficiente, maximizando el retorno y minimizando los riesgos. Por lo tanto, desarrollar los pozos en **Región 0** es la decisión más estratégica y rentable.

